In [1]:
!pip install -q kaggle
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"mitanshukhandal","key":"ce8cba9dc61f545062c9e0a85cab9a44"}'}

In [2]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [3]:
!kaggle datasets download -d andrezaza/clapper-massive-rotten-tomatoes-movies-and-reviews

Dataset URL: https://www.kaggle.com/datasets/andrezaza/clapper-massive-rotten-tomatoes-movies-and-reviews
License(s): CC0-1.0
 97% 147M/152M [00:01<00:00, 128MB/s]
100% 152M/152M [00:01<00:00, 105MB/s]


In [4]:
!unzip clapper-massive-rotten-tomatoes-movies-and-reviews.zip

Archive:  clapper-massive-rotten-tomatoes-movies-and-reviews.zip
  inflating: rotten_tomatoes_movie_reviews.csv  
  inflating: rotten_tomatoes_movies.csv  


In [5]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


In [6]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
import string
from nltk.stem import WordNetLemmatizer
import re
# Downloading stopwords and wordnet
nltk.download('stopwords', quiet=True)
nltk.download('wordnet', quiet=True)
#already defining which columns to keep in both movies and reviews
#how do i know about colums?->i tried creating this model before but i couldnt
movies_cols = ['id', 'title', 'genre', 'originalLanguage']
reviews_cols = ['id', 'reviewText']
#load dataset
movies_data = pd.read_csv('/content/rotten_tomatoes_movies.csv', usecols=movies_cols)
reviews_data = pd.read_csv('/content/rotten_tomatoes_movie_reviews.csv', usecols=reviews_cols)

# Merge datas based on id since its common
merged_data = pd.merge(movies_data, reviews_data, on='id')

# Cleaning up useless memory
#i know this because i tried this in morning but at that model broke so already using optimized codes
del movies_data, reviews_data

# Remove rows with nan values of our columns
merged_data.dropna(subset=['reviewText', 'id','genre','originalLanguage'], inplace=True)
#setting us stopwords and lemmatizer to what we installed earlier(lemmatizing is equivalent of stemming)
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
#defining a function to preprocess text
def preprocess_text(text):
  try:

    if not isinstance(text, str):
      return ""

    # Converting to lowercase
    text = text.lower()

    # Removing punctuation
    text = re.sub(r'[^\w\s]', '', text)

    # Splitting the text into tokens
    tokens = text.split()

    # Removing stop words and lemmatizing
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]

    return ' '.join(tokens)
  except:
    return ""
merged_data['processed_reviewText']=''
# Process text in chunks to save memory
chunk_size = 10000
for i in range(0, len(merged_data), chunk_size):
  print(f"Processing chunk {i//chunk_size+1}/{len(merged_data)//chunk_size+1}")
  chunk = merged_data.iloc[i:i+chunk_size].copy()
  chunk['processed_reviewText'] = chunk['reviewText'].apply(preprocess_text)

  # Update original dataframe
  merged_data.loc[chunk.index, 'processed_reviewText'] = chunk['processed_reviewText']

  # Print progress
  print(f"Processed {min(i+chunk_size, len(merged_data))} out of {len(merged_data)} reviews")
#THis cell is optimized cell using gpt. what was done earlier was something like this but was not optimized.The gist was the same

Processing chunk 1/138
Processed 10000 out of 1371079 reviews
Processing chunk 2/138
Processed 20000 out of 1371079 reviews
Processing chunk 3/138
Processed 30000 out of 1371079 reviews
Processing chunk 4/138
Processed 40000 out of 1371079 reviews
Processing chunk 5/138
Processed 50000 out of 1371079 reviews
Processing chunk 6/138
Processed 60000 out of 1371079 reviews
Processing chunk 7/138
Processed 70000 out of 1371079 reviews
Processing chunk 8/138
Processed 80000 out of 1371079 reviews
Processing chunk 9/138
Processed 90000 out of 1371079 reviews
Processing chunk 10/138
Processed 100000 out of 1371079 reviews
Processing chunk 11/138
Processed 110000 out of 1371079 reviews
Processing chunk 12/138
Processed 120000 out of 1371079 reviews
Processing chunk 13/138
Processed 130000 out of 1371079 reviews
Processing chunk 14/138
Processed 140000 out of 1371079 reviews
Processing chunk 15/138
Processed 150000 out of 1371079 reviews
Processing chunk 16/138
Processed 160000 out of 1371079 re

In [7]:
#lets check what we got in preprocessed data
merged_data

,id,title,genre,originalLanguage,reviewText,processed_reviewText
0,love_lies,"Love, Lies",Drama,Korean,Though let down by its routine love triangle n...,though let routine love triangle narrative lov...
1,love_lies,"Love, Lies",Drama,Korean,"While not perfect, Love, Lies is a worthy disc...",perfect love lie worthy discovery time culture...
2,adrift_2018,Adrift,"Adventure, Drama, Romance",English,This is nowhere near the level of other great ...,nowhere near level great survival stories44 en...
3,adrift_2018,Adrift,"Adventure, Drama, Romance",English,"Adrift somehow survives the late reveal, and t...",adrift somehow survives late reveal awkward st...
4,adrift_2018,Adrift,"Adventure, Drama, Romance",English,"For almost the entire runtime, the only people...",almost entire runtime people onscreen woodley ...
...,...,...,...,...,...,...
1469538,the-human-body,The Human Body,Documentary,English,"From beginning to end, The Human Body is a rar...",beginning end human body rare visual aural treat
1469539,the-human-body,The Human Body,Documentary,English,The impression left is that of a series of sli...,impression left series slide lecturer flashing...
1469540,the-human-body,The Human Body,Documentary,English,"Though plenty of room for improvement exists, ...",though plenty room improvement exists human bo...
1469541,the-human-body,The Human Body,Documentary,English,My guess is that The Human Body will inspire m...,guess human body inspire enterprising young me...


In [8]:
# !pip install transformers
# from transformers import AutoTokenizer, AutoModel
# from tqdm import tqdm
# import torch
# import numpy as np
# # Load pre-trained DistilBERT tokenizer and model
# tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
# model = AutoModel.from_pretrained("distilbert-base-uncased").to(device)
# # Process in batches for memory eficiency
# batch_size = 16
# #lets create embedding list
# embeddings_list = []
# # Creating batches and processing
# for i in tqdm(range(0, len(merged_data), batch_size)):
#   batch_texts = merged_data['processed_reviewText'][i:i+batch_size].tolist()
#   batch_texts = [text for text in batch_texts if text is not None and isinstance(text,str)]
#   if not batch_texts:
#     continue
#   # Tokenizing batch
#   inputs = tokenizer(batch_texts, return_tensors="pt", truncation=True,
#                       padding=True, max_length=512)

#   # Moving inputs to GPU
#   inputs = {k: v.to(device) for k, v in inputs.items()}

#   # Processing batch
#   with torch.no_grad():  # Disable gradient calculation
#       outputs = model(**inputs)

#   # Getting embeddings (mean pooling)
#   batch_embeddings = outputs.last_hidden_state.mean(dim=1).cpu().numpy()
#   embeddings_list.extend(batch_embeddings)

# # Convert to numpy array
# all_embeddings = np.array(embeddings_list)

# # Save embeddings to file instead of DataFrame
# np.save('review_embeddings.npy', all_embeddings)

# # Store reference in DataFrame
# merged_data['embedding_index'] = range(len(merged_data))


In [9]:
# #now we need a sentiment analyser
# #we can use VADER and Bert but using pipeline is more easy to implement
# from transformers import pipeline
# classification = pipeline("sentiment-analysis")

In [10]:
# sentiments=[]
# batch_size = 16
# for i in tqdm(range(0,len(merged_data),batch_size)):
#   batch = merged_data['processed_reviewText'][i:i+batch_size].tolist()
#   batch = [text for text in batch if text is not None and isinstance(text,str)]

#   batch_sentiments = classification(batch)
#   for r in batch_sentiments:
#     label = r['label']
#     if label=='POSITIVE':
#       sentiments.append('positive')
#     elif label=='NEGATIVE':
#       sentiments.append('negative')
#     else:
#       sentiments.append('neutral')

# sentiments.extend([None]*(len(merged_data)-len(sentiments)))
# merged_data['sentiment'] = sentiments

#couldnt run the model gpu limit excedded so had to use chatgpt for simple classification as not much time left

In [11]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import numpy as np
from tqdm import tqdm

# Choose a smaller model for efficiency
model_name = "distilbert-base-uncased-finetuned-sst-2-english"

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# CPU optimizations
model.eval()  # Set to evaluation mode
torch.set_num_threads(4)  # Limit CPU threads, adjust based on your system

# Efficient batch processing function
def process_reviews_in_batches(texts, batch_size=16, max_length=128):

  results = []

    # Process in batches with a progress bar
  for i in tqdm(range(0, len(texts), batch_size)):


    batch_texts = texts[i:i+batch_size]

    # Tokenize with truncation and padding
    encoded = tokenizer(


        batch_texts,
        truncation=True,
        padding='max_length',
        max_length=max_length,
        return_tensors='pt'
      )

  # Move to CPU explicitly
    input_ids = encoded['input_ids'].to('cpu')
    attention_mask = encoded['attention_mask'].to('cpu')

    # Disable gradient calculation for inference
    with torch.no_grad():

      outputs = model(input_ids, attention_mask=attention_mask)

        # Get predictions
    scores = torch.nn.functional.softmax(outputs.logits, dim=1)
    predictions = scores.detach().cpu().numpy()

        # Process each prediction in the batch
    for j, pred in enumerate(predictions):
        # Get the class with highest probability
        label_id = np.argmax(pred)
        confidence = pred[label_id]

        # Map to sentiment label
        sentiment = model.config.id2label[label_id]

        results.append({
            'index': i+j,
            'sentiment': sentiment,
            'confidence': float(confidence)
        })

  return results

# Sample a manageable subset of your data for speed
# Adjust sample_size based on your available RAM and CPU
sample_size = 5000  # Increase this if your system can handle it
sample_data = merged_data.sample(sample_size, random_state=42)

# Process the sampled data
print(f"Processing {sample_size} reviews...")
sentiment_results = process_reviews_in_batches(
    sample_data['processed_reviewText'].tolist(),
    batch_size=16,  # Small batch size for CPU
    max_length=128  # Limit sequence length for speed
)

# Convert results to DataFrame and merge with original data
sentiment_df = pd.DataFrame(sentiment_results)
sample_data = pd.merge(
    sample_data.reset_index(),
    sentiment_df,
    left_index=True,
    right_on='index'
)

# Map binary sentiment to three classes (positive, neutral, negative)
def map_to_three_classes(label, confidence):

  confidence_threshold = 0.75

  if label == "POSITIVE":

    if confidence > confidence_threshold:
        return 2  # Positive
    else:
        return 1  # Neutral leaning positive
  else:  # NEGATIVE
      if confidence > confidence_threshold:
          return 0  # Negative
      else:
          return 1  # Neutral leaning negative

# Apply the mapping
sample_data['sentiment_class'] = sample_data.apply(
  lambda row: map_to_three_classes(row['sentiment'], row['confidence']),
  axis=1
)

# Check distribution of sentiment classes
print("Sentiment Distribution:")
print(sample_data['sentiment_class'].value_counts())

# Save the model for future use
model_path = './sentiment_model'
tokenizer.save_pretrained(model_path)
model.save_pretrained(model_path)

print(f"Model saved to {model_path}")

# Create a simple function for analyzing new reviews
def analyze_new_review(review_text):
# Preprocess
  processed = preprocess_text(review_text)

  # Tokenize
  inputs = tokenizer(

      processed,
      return_tensors="pt",
      truncation=True,
      max_length=128
    )

    # Get prediction
  with torch.no_grad():
      outputs = model(**inputs)

  scores = torch.nn.functional.softmax(outputs.logits, dim=1)
  label_id = torch.argmax(scores, dim=1).item()
  confidence = scores[0][label_id].item()

  # Map to sentiment and class
  sentiment = model.config.id2label[label_id]
  sentiment_class = map_to_three_classes(sentiment, confidence)

  sentiment_map = {0: "Negative", 1: "Neutral", 2: "Positive"}

  return {

      "sentiment": sentiment_map[sentiment_class],
      "confidence": confidence,
      "review": review_text
    }


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Processing 5000 reviews...


100%|██████████| 313/313 [17:37<00:00,  3.38s/it]


Sentiment Distribution:
sentiment_class
2    2770
0    2027
1     203
Name: count, dtype: int64
Model saved to ./sentiment_model
{'sentiment': 'Positive', 'confidence': 0.9998903274536133, 'review': 'This movie was absolutely fantastic. I loved every minute of it!'}


In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV
from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostClassifier

# Prepare data
X = sample_data['processed_reviewText']
y = sample_data['sentiment_class']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create TF-IDF features
tfidf = TfidfVectorizer(max_features=100)
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)


svm = LinearSVC(random_state=42)
adaboost_svm = AdaBoostClassifier(
    estimator=svm,
    n_estimators=50,  # Number of weak learners (adjust as needed)
    learning_rate=0.1,  # Learning rate (adjust as needed)
    random_state=42  # For reproducibility
)
svm_calibrated = CalibratedClassifierCV(adaboost_svm)
svm_calibrated.fit(X_train_tfidf, y_train)

# Evaluate
from sklearn.metrics import classification_report
y_pred = svm_calibrated.predict(X_test_tfidf)
print(classification_report(y_test, y_pred,zero_division=1))

# Create an analysis function
def analyze_with_svm(text):
  processed = preprocess_text(text)
  features = tfidf.transform([processed])
  prediction = svm_calibrated.predict(features)[0]
  probabilities = svm_calibrated.predict_proba(features)[0]
  confidence = probabilities[prediction]

  sentiment_map = {0: "Negative", 1: "Neutral", 2: "Positive"}
  sentiment = sentiment_map[prediction]

  return {

      "sentiment": sentiment,
      "confidence": confidence,
      "review": text
    }

# Save the SVM model and TF-IDF vectorizer
import pickle
with open('tfidf_vectorizer.pkl', 'wb') as f:
  pickle.dump(tfidf, f)
with open('svm_model.pkl', 'wb') as f:
  pickle.dump(svm_calibrated, f)

KeyboardInterrupt: 